# It works importing Regression_variables!! Implement this later

In [1]:
#import Regression_variables from same directory
from regression_variable_morgan_comparison import *

# Set path to this notebook
HERE = Path(_dh[-1])
DATA = HERE / "data"

# compounds is name of filtered dataframe

KeyboardInterrupt: 

In [ ]:
svm_rand = svm.SVR()
svm_rand.fit(np.array(list((x_train_rand))).astype(float), y_train_rand)
y_pred_svm_rand = svm_rand.predict(np.array(list((x_test_rand))).astype(float))

svm_strat = svm.SVR()
svm_strat.fit(np.array(list((x_train_strat))).astype(float), y_train_strat)
y_pred_svm_strat = svm_strat.predict(np.array(list((x_test_strat))).astype(float))

svm_hi = svm.SVR()
svm_hi.fit(np.array(list((x_train_hi))).astype(float), y_train_hi)
y_pred_svm_hi = svm_hi.predict(np.array(list((x_test_hi))).astype(float))

svm_noise = svm.SVR()
svm_noise.fit(np.array(list((x_train_noise))).astype(float), y_train_noise)
y_pred_svm_noise = svm_noise.predict(np.array(list((x_test_noise))).astype(float))

svm_fur = svm.SVR()
svm_fur.fit(np.array(list((x_train_fur))).astype(float), y_train_fur)
y_pred_svm_fur = svm_fur.predict(np.array(list((x_test_fur))).astype(float))

In [ ]:
mae_svm_rand = mean_absolute_error(y_test_rand, y_pred_svm_rand)
mse_svm_rand = mean_squared_error(y_test_rand, y_pred_svm_rand)
r2_svm_rand = r2_score(y_test_rand, y_pred_svm_rand)

mae_svm_strat = mean_absolute_error(y_test_strat, y_pred_svm_strat)
mse_svm_strat = mean_squared_error(y_test_strat, y_pred_svm_strat)
r2_svm_strat = r2_score(y_test_strat, y_pred_svm_strat)

mae_svm_hi = mean_absolute_error(y_test_hi, y_pred_svm_hi)
mse_svm_hi = mean_squared_error(y_test_hi, y_pred_svm_hi)
r2_svm_hi = r2_score(y_test_hi, y_pred_svm_hi)

mae_svm_noise = mean_absolute_error(y_test_noise, y_pred_svm_noise)
mse_svm_noise = mean_squared_error(y_test_noise, y_pred_svm_noise)
r2_svm_noise = r2_score(y_test_noise, y_pred_svm_noise)

mae_svm_fur = mean_absolute_error(y_test_fur, y_pred_svm_fur)
mse_svm_fur = mean_squared_error(y_test_fur, y_pred_svm_fur)
r2_svm_fur = r2_score(y_test_fur, y_pred_svm_fur)

In [ ]:
#Creat a table that shows the performance of svm on the different splits displaying mae, mse and r2 values
split_scores_svm_initial = {'Split': ['Random', 'Stratified pIC50', 'Hierarchical Clustering', 'UMAP Noise', 'UMAP Furthest Cluster'],
                                'MAE': [mae_svm_rand, mae_svm_strat, mae_svm_hi, mae_svm_noise, mae_svm_fur],
                                'MSE': [mse_svm_rand, mse_svm_strat, mse_svm_hi, mse_svm_noise, mse_svm_fur],
                                'R2': [r2_svm_rand, r2_svm_strat, r2_svm_hi, r2_svm_noise, r2_svm_fur]}

split_scores_svm_initial_df = pd.DataFrame(split_scores_svm_initial)
split_scores_svm_initial_df.set_index("Split", inplace=True)
#save df to csv in data file
split_scores_svm_initial_df.to_csv(DATA / "split_scores_svm_initial.csv")

print(split_scores_svm_initial_df)

                              MAE       MSE        R2
Split                                                
Random                   0.439571  0.394999  0.463319
Stratified pIC50         0.517414  0.483615  0.447697
Hierarchical Clustering  0.708966  0.765479  0.265783
UMAP Noise               0.840197  0.962615 -0.196211
UMAP Furthest Cluster    0.814923  0.905189 -0.193262


In [ ]:
def objective(trial):

    # Invoke suggest methods of a Trial object to generate hyperparameters.
    regressor_name = trial.suggest_categorical('classifier', ['SVR', 'RandomForest'])
    if regressor_name == 'SVR':
        svr_c = trial.suggest_float('svr_c', 1e-10, 1e10, log=True)
        regressor_obj = sklearn.svm.SVR(C=svr_c)
    else:
        rf_max_depth = trial.suggest_int('rf_max_depth', 2, 32)
        regressor_obj = sklearn.ensemble.RandomForestRegressor(max_depth=rf_max_depth)

    X, y = sklearn.datasets.fetch_california_housing(return_X_y=True)
    X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y, random_state=0)

    regressor_obj.fit(X_train, y_train)
    y_pred = regressor_obj.predict(X_val)

    error = sklearn.metrics.mean_squared_error(y_val, y_pred)

    return error  # An objective value linked with the Trial object.

In [ ]:
study_svm_rand = optuna.create_study(direction="maximize")  # Create a new study.
study_svm_rand.optimize(objective, n_trials=100)  # Invoke optimization of the objective function.

[I 2024-07-31 10:40:39,129] A new study created in memory with name: no-name-c7030db6-1f1d-4504-be85-d4c87e9ecebb
[I 2024-07-31 10:41:20,250] Trial 0 finished with value: 1.3824372958226088 and parameters: {'classifier': 'SVR', 'svr_c': 0.005227992598399137}. Best is trial 0 with value: 1.3824372958226088.
